## 找到動畫瘋如何表示集數
一部動畫的id會在參數中以`sn`表示  
以[https://ani.gamer.com.tw/animeVideo.php?sn=34142](https://ani.gamer.com.tw/animeVideo.php?sn=34142)為例  
`sn`是34142

In [27]:
sn = '34142'

## 觀察network流量
先看到了`playlist.m3u8`，用ffplay播放

In [2]:
! ffplay -i https://bahamut.akamaized.net/ad/welcome_to_anigamer/playlist.m3u8

ffplay version N-107626-g1368b5a725-20220731 Copyright (c) 2003-2022 the FFmpeg developers
  built with gcc 12.1.0 (crosstool-NG 1.25.0.55_3defb7b)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-libdav1d --enable-libdavs2 --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libass --enable-libbluray --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librist --enable-libtheora --enable-libvpx --enable-libwebp --enable-lv2 --

但顯然這個播放清單是歡迎畫面的，不是動畫本體

觀察有可能的API
- `getdeviceid.php`
- `video.php`
- `token.php`

## 初始化工作階段
後續會呼叫的API有一部分會認cookie，且都會認header，這些在開發的時候會一個一個測試，但是這邊就先寫結論方便展示  
header會認以下幾個：
- user-agent
- referer
- origin

In [26]:
import requests

header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.70', 'referer': 'https://ani.gamer.com.tw/animeVideo.php', 'origin': 'https://ani.gamer.com.tw'}
session = requests.session()
session.headers.update(header)
print(session.cookies)

<RequestsCookieJar[]>


## 觀看廣告
沒有買會員的話就需要看30秒廣告才能開始看動畫，但這個30秒怎麼計數的呢？  
開始廣告之後會發現一個API呼叫
- `videoCastcishu.php?s=...&sn=37780`

廣告結束後也有一個呼叫
- `videoCastcishu.php?s=...&sn=37780&ad=end`

如果是客戶端計數，直接無視就好，如果是伺服器端計數，就乖乖等

### 取得廣告的ID
生成廣告ID是直接從原始碼幹出來用python復刻的，可以當作魔法

In [29]:
import functions
ad_data = functions.get_major_ad()
session.cookies.update(ad_data['cookie'])
print(ad_data)

{'adsid': '218406', 'cookie': {'ckBahaAd': '------9---------'}}


### 模擬看廣告
將廣告ID送出去，假裝自己開始看了廣告，計時之後再送看完了的封包  
廣告原本是30秒，但實際上伺服器端計時約25秒

In [30]:
import time

print('start ad')
session.get(f'https://ani.gamer.com.tw/ajax/videoCastcishu.php?s={ad_data["adsid"]}&sn={sn}')
ad_countdown = 25
for countdown in range(ad_countdown):
    print(f'ad {ad_countdown - countdown}s remaining', end='\r')
    time.sleep(1)

session.get(f'https://ani.gamer.com.tw/ajax/videoCastcishu.php?s={ad_data["adsid"]}&sn={sn}&ad=end')
print('\nend ad')

start ad
ad 1s remainingg
end ad


In [31]:
print(session.cookies)

<RequestsCookieJar[<Cookie ckBahaAd=------9--------- for />, <Cookie ANIME_SIGN=0479f7f8228feea74f144d5b273fc97d279f6e3c2d84daf5661c190e for .gamer.com.tw/>, <Cookie nologinuser=047a066ce24418fbf68a780436951f2f8b21721032f10244661c190e8807 for .gamer.com.tw/>, <Cookie __cf_bm=dFyh.AcUOICFNv558ltx02SkJwPWIfR9gNmEt2LIEUw-1713117454-1.0.1.1-t1q6AM.H9oBLL344jz5t9j8ZLIirIqyczTSGw3K0eNURUl3IXLCnhOIAe7Pk_aAXxL6hyyRCMFsrTRfxTzjR.Q for .gamer.com.tw/>]>


## 取得播放清單
可以觀察到一個叫做`m3u8.php`的API裡面取得了m3u8的網址，例如下面這樣
- https://ani.gamer.com.tw/ajax/m3u8.php?sn=37780&device=0052f38c1634575e5e81f13b1daef008c6ae921bc6415a7d660cea1c4571

request參數裡面包含了deviceid，經過測試之後這是必要的參數  
而`m3u8.php`傳回內容是一個m3u8的網址

### 取得devideid

In [32]:
import json

deviceid_res = session.get('https://ani.gamer.com.tw/ajax/getdeviceid.php')
deviceid_res.raise_for_status()
deviceid = json.loads(deviceid_res.text)['deviceid']
print(deviceid)

0536162c74513b443ebbfcfa3f355d268c1d9a4b316175fd661c19400136


In [ ]:
print(session.cookies)

### 取得播放清單網址

In [11]:
m3u8_php_res = session.get(f'https://ani.gamer.com.tw/ajax/m3u8.php?sn={sn}&device={deviceid}')
m3u8_php_res.raise_for_status()
try:
    playlist_basic_url = json.loads(m3u8_php_res.text)['src']
except Exception as ex:
    print(m3u8_php_res.text)
    print('failed to load m3u')
print(playlist_basic_url)

https://bahamut.akamaized.net/1132940b3ed86be8fe4f157ab12b64ded73badf5/playlist_guest.m3u8?hdnts=exp%3D1713038679%7Edata%3D3b88a9406764a3f%3A34142%3A0%3A1%3A68686832%7Eacl%3D%2F1132940b3ed86be8fe4f157ab12b64ded73badf5%2Fplaylist_guest.m3u8%21%2F1132940b3ed86be8fe4f157ab12b64ded73badf5%2F360p%2F%2A%7Ehmac%3Db1f9fa20f79e480bb2535962421f30f8567d1e34ee54f9682559b8c4f8afbce1


### 取得第一層播放清單
經由剝參數測試，這個資源會檢查header  
> 為何已經取得了m3u8卻不直接用ffmpeg下載？  

因為檢查的header有User-Agent、referer、origin，給ffmpeg時需要用`-header`參數傳遞，每個header一行，但是在windows環境下沒辦法傳遞換行字元至參數中，所以選擇自己寫下載的部分，在本機合併

In [12]:
import os

meta_base = os.path.dirname(playlist_basic_url) + '/'
playlist_basic_res = requests.get(playlist_basic_url, headers=header)
playlist_basic_res.raise_for_status()

print(playlist_basic_res.text)

#EXTM3U
#EXT-X-VERSION:3
#EXT-X-STREAM-INF:BANDWIDTH=400000,RESOLUTION=640x360
360p/hdntl=exp=1713121568~acl=%2f1132940b3ed86be8fe4f157ab12b64ded73badf5%2fplaylist_guest.m3u8!%2f1132940b3ed86be8fe4f157ab12b64ded73badf5%2f360p%2f*~data=hdntl,3b88a9406764a3f%3a34142%3a0%3a1%3a68686832~hmac=c2adae6d513fac52d448c12b2cc862ca0ffb4d95b37d37f97a0666297c42966a/chunklist_b400000.m3u8



### 分離出第二層播放清單網址
第一層播放清單中引用了第二層播放清單，未登入帳號只能看到360p的播放清單  
題外話，未登入一直以來都只能看360p，但以前其實未登入播放清單會給到720p，在前端動點手腳解除UI封印就可以播放了

In [13]:
playlist_basic = playlist_basic_res.text.split('\n')
resolutions_metadata = []
for i in range(len(playlist_basic)):
    line = playlist_basic[i]
    if line.startswith('#EXT-X-STREAM-INF'):
        resolutions_metadata.append({'info': line[18:], 'url': playlist_basic[i+1]})

In [14]:
print(json.dumps(resolutions_metadata, indent=4))

[
    {
        "info": "BANDWIDTH=400000,RESOLUTION=640x360",
        "url": "360p/hdntl=exp=1713121568~acl=%2f1132940b3ed86be8fe4f157ab12b64ded73badf5%2fplaylist_guest.m3u8!%2f1132940b3ed86be8fe4f157ab12b64ded73badf5%2f360p%2f*~data=hdntl,3b88a9406764a3f%3a34142%3a0%3a1%3a68686832~hmac=c2adae6d513fac52d448c12b2cc862ca0ffb4d95b37d37f97a0666297c42966a/chunklist_b400000.m3u8"
    }
]


### 列出可選的畫質
未登入的工作階段只能看到360p

In [15]:
for j in range(len(resolutions_metadata)):
    print('#%s: %s' % (j, resolutions_metadata[j]['info']))

#0: BANDWIDTH=400000,RESOLUTION=640x360


In [16]:
selected_resolution = int(input('select a resolution: '))
print(f'selected: #{selected_resolution}')

selected: #0


## 準備下載

### 準備下載用的資料夾

In [17]:
resolution_number = resolutions_metadata[selected_resolution]['info'].rsplit('=', 1)[1]
folder_name = f'{sn}_{resolution_number}'
os.makedirs(os.path.join('Download', folder_name), exist_ok=True)
os.chdir(os.path.join('Download', folder_name))

### 取得第二層播放清單
經由剝參數測試，這個資源也會檢查header

In [18]:
chunklist_url = meta_base + resolutions_metadata[selected_resolution]['url']
print(chunklist_url)

https://bahamut.akamaized.net/1132940b3ed86be8fe4f157ab12b64ded73badf5/360p/hdntl=exp=1713121568~acl=%2f1132940b3ed86be8fe4f157ab12b64ded73badf5%2fplaylist_guest.m3u8!%2f1132940b3ed86be8fe4f157ab12b64ded73badf5%2f360p%2f*~data=hdntl,3b88a9406764a3f%3a34142%3a0%3a1%3a68686832~hmac=c2adae6d513fac52d448c12b2cc862ca0ffb4d95b37d37f97a0666297c42966a/chunklist_b400000.m3u8


In [19]:
chunklist_res = requests.get(chunklist_url, headers=header)
print(chunklist_res.text)

#EXTM3U
#EXT-X-VERSION:3
#EXT-X-TARGETDURATION:10
#EXT-X-MEDIA-SEQUENCE:0
#EXT-X-KEY:METHOD=AES-128,URI="key_b400000.m3u8key",IV=0x7511ab57aafcb9bf3f5e18aa1f8ddb44
#EXTINF:10.427089,
media_b400000_0.ts
#EXTINF:10.427078,
media_b400000_1.ts
#EXTINF:10.427089,
media_b400000_2.ts
#EXTINF:10.427078,
media_b400000_3.ts
#EXTINF:10.427089,
media_b400000_4.ts
#EXTINF:10.427078,
media_b400000_5.ts
#EXTINF:10.427089,
media_b400000_6.ts
#EXTINF:10.427078,
media_b400000_7.ts
#EXTINF:10.427089,
media_b400000_8.ts
#EXTINF:10.427078,
media_b400000_9.ts
#EXTINF:10.427089,
media_b400000_10.ts
#EXTINF:10.427078,
media_b400000_11.ts
#EXTINF:10.427089,
media_b400000_12.ts
#EXTINF:10.427078,
media_b400000_13.ts
#EXTINF:10.427089,
media_b400000_14.ts
#EXTINF:10.427078,
media_b400000_15.ts
#EXTINF:10.427089,
media_b400000_16.ts
#EXTINF:10.427078,
media_b400000_17.ts
#EXTINF:10.427089,
media_b400000_18.ts
#EXTINF:10.427078,
media_b400000_19.ts
#EXTINF:10.427089,
media_b400000_20.ts
#EXTINF:10.427078,
media_b4

### 儲存m3u8至檔案

In [20]:
chunklist_filename = folder_name + '.m3u8'
with open(chunklist_filename, 'wb') as chunklist_file:
    for chunk in chunklist_res:
        chunklist_file.write(chunk)

### 分離出影片網址的base

In [21]:
chunks_base = meta_base + resolutions_metadata[selected_resolution]['url'].rsplit('/', 1)[0] + '/'
print(chunks_base)

https://bahamut.akamaized.net/1132940b3ed86be8fe4f157ab12b64ded73badf5/360p/hdntl=exp=1713121568~acl=%2f1132940b3ed86be8fe4f157ab12b64ded73badf5%2fplaylist_guest.m3u8!%2f1132940b3ed86be8fe4f157ab12b64ded73badf5%2f360p%2f*~data=hdntl,3b88a9406764a3f%3a34142%3a0%3a1%3a68686832~hmac=c2adae6d513fac52d448c12b2cc862ca0ffb4d95b37d37f97a0666297c42966a/


#### 初始化多執行緒下載模組
自己手刻的土炮多執行緒，但記得隊網站斯文一些才有禮貌，不要一傢伙榨乾對方頻寬

In [22]:
import multiple_thread_downloading
chunklist = chunklist_res.text.split('\n')
mtd_worker = multiple_thread_downloading.mtd(header, chunks_base, chunklist_res.text.count('.ts'))

#### 把.ts網址丟進queue
裡面有key的檔案，也順便丟進去下載

In [23]:
import re

for k in range(len(chunklist)):
    line = chunklist[k]
    if line.startswith('#EXTINF'):
        ts_name = chunklist[k+1]
        # push
        mtd_worker.push(ts_name)
    elif line.startswith('#EXT-X-KEY'):
        key_name = re.match('.*URI="(.*)".*$', line).group(1)
        # push
        mtd_worker.push(key_name)

### 等待下載完成

In [24]:
mtd_worker.join()

## 呼叫ffmpeg合併
key在m3u8裡面有寫到，ffmpeg會自己解開加密的部分

In [25]:
command = f'ffmpeg -allowed_extensions ALL -i {chunklist_filename} -c copy {folder_name}.mp4'
print(command)
os.system(command)

ffmpeg -allowed_extensions ALL -i 34142_640x360.m3u8 -c copy 34142_640x360.mp4


0